In [12]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching et_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s et_Scene"

In [186]:
bin_count = 8

# Array mit Eye-Tracking Daten ([x, y, t])
et_array = np.array([
    [-6, -3, 200],
    [-5, 1, 400],
    [-4, 0, 600],
    [-2, 2, 200],
    [-3, 2, 100],
    [-2, -1, 300],
    [-2, 2, 100],
    [-3, 3, 100],
    [-2, 2.5, 100],
    [-1.5, 2.5, 100],
    [-1, 2.5, 100],
    [-1, -3, 100],
    [-2, -3, 100],
    [-2, -2, 200],
    [-3, -3, 500],
    [-4, -3, 200],
])

et_pos_array = np.zeros(np.shape(et_array))
et_pos_array[:,:2], et_time_array = et_array[:,:2], et_array[:,2]


# calculates angle of vector v
def polar_angle(v):
    return np.arctan2(v[1], v[0])


# takes polar angle and returns bin indey and binned angle
def bin_angle(angle):
    for i, bin in enumerate(np.linspace(0, 2*PI, bin_count+1)):
        if bin > (angle + 2*PI) % (2*PI) - PI/bin_count:
            return i, bin

In [188]:
%%capture_video --path "animations/eye_tracking.mp4"
%%manim -qh --fps 60 $video_scene

class et_Scene(Scene):
    def construct(self):
        pol = PolarPlane(azimuth_units = "degrees", size = 4, azimuth_label_font_size = 33.6, azimuth_step = bin_count, azimuth_label_buff = 0.2).add_coordinates(r_values=[]).shift(4*RIGHT)
        self.add(pol)
        phi_vals = np.linspace(0, 2*PI, bin_count+1)
        r_vals = np.zeros_like(phi_vals)
        for i, pos in enumerate(et_pos_array):
            time = Text(f'{et_time_array[i]} ms').move_to(pos+0.5*UP).scale(0.5)
            dot = Dot(color = WHITE, radius = 0.1).move_to(pos)
            if i == 0:
                x_vals, y_vals = [r_vals[i1] * np.cos(phi_vals[i1]) for i1 in range(len(phi_vals))], [r_vals[i1] * np.sin(phi_vals[i1]) for i1 in range(len(phi_vals))]       
                graph0 = pol.plot_line_graph(x_values = x_vals, y_values = y_vals, stroke_color = YELLOW, add_vertex_dots = False, stroke_opacity = 0.5)        
                self.add(graph0)
                self.play(FadeIn(dot), Write(time))
                self.play(FadeOut(time), run_time = et_time_array[i] / 400)
            else:
                arrow = Arrow(start = et_pos_array[i-1], end = pos, color = RED, stroke_width = 1, buff = 0.2, tip_length = 0.15)
                index, binned_angel = bin_angle(polar_angle(pos - et_pos_array[i-1]))
                r_vals[index] += 1
                r_vals[-1] = r_vals[0] 
                plot_arrow = Arrow(start = [0, 0, 0], end = [r_vals[index] * np.cos(binned_angel) / 2, r_vals[index] * np.sin(binned_angel) / 2, 0], color = RED, buff = 0).shift(4*RIGHT)
                x_vals, y_vals = [r_vals[i1] * np.cos(phi_vals[i1]) for i1 in range(len(phi_vals))], [r_vals[i1] * np.sin(phi_vals[i1]) for i1 in range(len(phi_vals))]
                graph = pol.plot_line_graph(x_values = x_vals, y_values = y_vals, stroke_color = YELLOW, add_vertex_dots = False, stroke_opacity = 1)
                self.play(FadeIn(dot), Create(arrow), Write(time))
                self.play(TransformFromCopy(arrow, plot_arrow), Transform(graph0, graph))
                self.play(FadeOut(time), run_time = et_time_array[i] / 400)
        self.wait(3)